# Convert from ENVI to NetCDF

This is a pure GDAL/netCDF4 implementation of conversion from ENVI raster image for AVIRIS-NG. I will document a simpler workflow using `xarray` some other time.

<h2 id="tocheading">Table of Contents</h2>       
<br>
<div id="toc"></div>     

*This next cell calls a script to generate the TOC above. Ignore.*

In [1]:
%%javascript
$.getScript('scripts/tocgen.js')

<IPython.core.display.Javascript object>

## Review

Define some functions that include all of the read/write and spatial logic that was covered in [the first notebook](0_read_write_with_python_gdal.ipynb):     

In [2]:
def get_global_attributes(hdr):
    """Rudimentary parser for ENVI header file. Improve."""
    
    with open(hdr,'r') as f:
        header = [ln.strip() for ln in f.readlines()]

        global_atts = dict(
            description = header[2][:-1],
            samples = header[3].split("=")[1].strip(),
            lines = header[4].split("=")[1].strip(),
            bands = header[5].split("=")[1].strip(),
            data_type = header[8].split("=")[1].strip(),
            source_type = header[7].split("=")[1].strip(),
            interleave = header[9].split("=")[1].strip(),
            byte_order = header[10].split("=")[1].strip(),
            map_info = header[11].split("=")[1].strip(),
            wavelength_units = header[13].split("=")[1].strip(),
            missing_value = header[24].split("=")[1].strip())
        global_atts["Conventions"] = "CF-1.6"
    
    return(global_atts)
    

def get_shape(ds):
    """Get number of bands, columns, and rows in raster."""
    
    return(ds.RasterCount,  # band count
           ds.RasterXSize,  # col count
           ds.RasterYSize)  # row count


def get_xy_arrays(ds):
    """Generate two 1d x,y coordinate arrays."""
    
    # get raster shape
    bands, cols, rows = get_shape(ds)
    
    # get the raster geotransform as its component parts
    xmin, xres, xrot, ymax, yrot, yres = ds.GetGeoTransform()
    
    # generate coordinate arrays
    xarr = np.array([xmin+i*xres for i in range(0,cols)])
    yarr = np.array([ymax+i*yres for i in range(0,rows)])

    return(xarr, yarr)
    
    
def get_proj(ds):
    """Returns the osr spatial reference object and proj4."""
    
    native_srs = osr.SpatialReference()
    native_srs.ImportFromWkt(ds.GetProjection())
    proj4 = native_srs.ExportToProj4()
    
    return(native_srs, proj4)
    
    
def get_latlon_arrays(ds):
    """Generate two 2d lat,lon coordinate arrays."""
    
    native_srs, proj4 = get_proj(ds)
    
    inproj = Proj(proj4)
    outproj = Proj(init="epsg:4326")
    
    xarr, yarr = get_xy_arrays(ds)
    lon, lat = transform(inproj, outproj, xarr[0], yarr[0])

    # get two 2d arrays of lats and lons
    xarr2d, yarr2d = np.meshgrid(xarr, yarr)

    # flatten both arrays and transform
    lonarr, latarr = transform(
        inproj,                   # input raster srs
        outproj,                  # output raster srs
        xarr2d.flatten(),         # flat 2d array of x coordinates
        yarr2d.flatten())         # flat 2d array of y coordinates

    # return flat arrays to shape of input raster
    lonarr2d = lonarr.reshape(xarr2d.shape)
    latarr2d = latarr.reshape(yarr2d.shape)
    
    return(lonarr2d, latarr2d)

## Workflow

### Imports and example file
Import requirements. Minimal packages:

In [3]:
import numpy as np
import netCDF4 as nc4
from osgeo import gdal, osr
from pyproj import Proj, transform

from scripts.prog import log_progress # optional

Example reflectance file:

In [4]:
img = 'data/ang20180814t224053_rfl_v2r2/ang20180814t224053_corr_v2r2_img'
hdr = 'data/ang20180814t224053_rfl_v2r2/ang20180814t224053_corr_v2r2_img.hdr'

ds = gdal.Open(img)

### Creating a new netCDF dataset

The remaining the steps use the basic functionality of the Python interface to *libnetcdf* (maintained by Unidata).

#### Open in write mode
Open a netCDF4.Dataset for writing:

In [5]:
nc = nc4.Dataset("output/test.nc", "w")  

nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): 
    variables(dimensions): 
    groups: 

#### Add coordinate dimensions
Use the `get_shape()` function defined above to get the shape of the raster image and add dimensions **band, y, x**:

In [6]:
bands, cols, rows = get_shape(ds)           # get image shape

band_dim = nc.createDimension('band')       # add band dim
y_dim = nc.createDimension('y', size=rows)  # add y dim
x_dim = nc.createDimension('x', size=cols)  # add x dim

nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): band(0), y(4207), x(637)
    variables(dimensions): 
    groups: 

#### Add coordinate variables
Use the `get_xy_arrays()` function defined above to get the two 1d arrays of x and y coordinates, then add the x and y variables with proper CF attributes:

In [7]:
xarr, yarr = get_xy_arrays(ds)

y_var = nc.createVariable('y', 'float32', ('y'))
y_var.units = "m"
y_var.standard_name = "projection_y_coordinate"
y_var.long_name = "y coordinate of projection"
y_var[:] = yarr

x_var = nc.createVariable('x', 'float32', ('x'))
x_var.units = "m"
x_var.standard_name = "projection_x_coordinate"
x_var.long_name = "x coordinate of projection"
x_var[:] = xarr

nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): band(0), y(4207), x(637)
    variables(dimensions): float32 y(y), float32 x(x)
    groups: 

Now add a variable to store the band numbers:

In [8]:
bandnum_var = nc.createVariable('band', 'short', ('band'))
bandnum_var.long_name = "AVIRIS-NG band/channel number"
bandnum_var[:] = list(range(1, bands+1))

#### Grid mapping
Make a function to collect some attributes for the `crs` variable:

In [9]:
def get_crs_attributes(ds):
    """Collect projection parameters for CRS variable."""
    
    native_srs, proj4 = get_proj(ds)
    
    crs_atts = {
        par: native_srs.GetProjParm(par) for par in [
            "scale_factor_at_central_meridian",
            "longitude_of_central_meridian",
            "latitude_of_projection_origin",
            "false_easting",
            "false_northing"]}
    
    crs_atts["utm_zone"] = native_srs.GetUTMZone()
    crs_atts["proj4"] = proj4
    crs_atts["crs_wkt"] = ds.GetProjection()
    
    return(crs_atts)

Test the new function and add the `crs` variable to the file:

In [10]:
crs_atts = get_crs_attributes(ds)

crs_var = nc.createVariable('crs', 'short')
crs_var.grid_mapping_name = "transverse_mercator"
for att,val in crs_atts.items():
    crs_var.setncattr(att,val)

print(crs_var)

<class 'netCDF4._netCDF4.Variable'>
int16 crs()
    grid_mapping_name: transverse_mercator
    scale_factor_at_central_meridian: 0.0
    longitude_of_central_meridian: 0.0
    latitude_of_projection_origin: 0.0
    false_easting: 500000.0
    false_northing: 0.0
    utm_zone: 3
    proj4: +proj=utm +zone=3 +datum=WGS84 +units=m +no_defs 
    crs_wkt: PROJCS["UTM Zone 3, Northern Hemisphere",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-165],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["Meter",1]]
unlimited dimensions: 
current shape = ()
filling on, default _FillValue of -32767 used



#### Latitude and longitude variables
The projected netCDF specification (CF) requires that 2-dimensional arrays be stored for both latitude and longitude. 

Use the function defined above `get_latlon_arrays()` to get the arrays and add the **lat, lon** variables:

In [11]:
latarr2d, lonarr2d = get_latlon_arrays(ds)

lat_var = nc.createVariable('lat', 'float32', ('y', 'x'))
lat_var.units = "degrees_north"
lat_var.standard_name = "latitude"
lat_var.long_name = "latitude coordinate"
lat_var[:,:] = latarr2d

lon_var = nc.createVariable('lon', 'float32', ('y', 'x'))
lon_var.units = "degrees_east"
lon_var.standard_name = "longitude"
lon_var.long_name = "longitude coordinate"
lon_var[:,:] = lonarr2d

nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): band(425), y(4207), x(637)
    variables(dimensions): float32 y(y), float32 x(x), int16 band(band), int16 crs(), float32 lat(y,x), float32 lon(y,x)
    groups: 

#### Wavelength and reflectance variables

Add two final variables to the file without setting values:
1. a 1d variable (band dimension) to store the wavelength of the center of each band (floats)
2. a 3d variable (band, y, x) to store the raster

In [12]:
# add wavelength variable
bandwav_var = nc.createVariable('band_wavelength', 'float32', ('band'))
bandwav_var.units = "nanometers"
bandwav_var.long_name = "wavelength of band center"

# add reflectance variable
refl_var = nc.createVariable('reflectance', 'float32', ('y', 'x', 'band'))
refl_var.units = "unitless"
refl_var.coordinates = "lon lat"
refl_var.grid_mapping = "crs"
refl_var.standard_name = "reflectance"
refl_var.long_name = "atmospherically corrected surface reflectance"

nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): band(425), y(4207), x(637)
    variables(dimensions): float32 y(y), float32 x(x), int16 band(band), int16 crs(), float32 lat(y,x), float32 lon(y,x), float32 band_wavelength(band), float32 reflectance(y,x,band)
    groups: 

**Write buffered data to the output file:**

In [13]:
nc.sync()

### Read from ENVI and write to netCDF in a loop

This last section demonstrates how to read from the ENVI raster image and write to the output netCDF in a loop (one band at a time) such that we never have more than one band of the image in memory at a time. 

#### Parsing band metadata from ENVI image
This snippet is imperfect. It may not work for ENVI raster images that aren't generated by the standard AVIRIS-NG processing logic at JPL. Hard to say. But, here we are parsing the band number:center wavelength pairs accessible via GDAL through the band-level metadata:

In [14]:
import collections

# make a dictionary of band number, band center pairs
wl = {}
for n, w in ds.GetMetadata().items():
    if "Band" in n:
        wl[int(n.split("_")[1])] = float(w.split(" ")[0])

# convert to ordered dict and sort
wl = collections.OrderedDict(sorted(wl.items()))
wl[1]

376.85999999999996

#### Writing in a loop
Iterate over the length of the band dimension, writing each raster band to variable `reflectance` and its wavelength to variable `band_wavelength`:

In [15]:
for b in log_progress(range(1, nc.dimensions['band'].size + 1)):
    #print("Writing band " + str(b) + " ... ")
    
    # get band b from input raster image
    barr = ds.GetRasterBand(b).ReadAsArray()
    
    # set all -9999. to numpy.nan
    barr[barr==-9999.] = np.nan
    
    # update the nc reflectance array for band b
    nc.variables['reflectance'][:, :, b-1] = barr
    
    # update the nc wavelength array for band b
    nc.variables['band_wavelength'][b-1] = wl[b]
    
    # write changes to disk
    nc.sync()
    
nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): band(425), y(4207), x(637)
    variables(dimensions): float32 y(y), float32 x(x), int16 band(band), int16 crs(), float32 lat(y,x), float32 lon(y,x), float32 band_wavelength(band), float32 reflectance(y,x,band)
    groups: 

#### Add global attributes

Use the function defined above `get_global_attributes(<header_file.hdr>)` to get a dictionary of global attributes and add them to the netCDF in a loop:

In [16]:
# get global attributes
global_atts = get_global_attributes(hdr)

# add global attributes
for att,val in global_atts.items():
    nc.setncattr(att,val)

#### Close the file

Do not forget to close the file! Python will close it during clean up when your session ends, but it's best to close manually:

In [17]:
nc.close()